In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from dpt.plmodels import InteriorNetDPT
from data.InteriorNetDataset import InteriorNetDataset
from data.metrics import SILog, DepthMetrics

In [2]:
# path = 'christh9-pvc/train-logs/finetune/grads'
# # ddp, no_ddp = os.listdir(path)
# # ddp = torch.load(os.path.join(path,ddp))
# # no_ddp = torch.load(os.path.join(path, no_ddp))
# # for i in range(len(ddp)):
# #     print(torch.all(ddp[i] == no_ddp[i]))
# sorted(os.listdir(path))
# horovod_1_path = os.path.join(path,'grads-single-horovod-0.pt')

In [3]:
# horovod_single = torch.load(single_horovod)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device name: {torch.cuda.get_device_name(0) if device != "cpu" else "cpu"}')

device name: A40


In [3]:
# k8s paths
k8s = True
k8s_repo = r'opt/repo/dynamic-inference'
k8s_pvc = r'christh9-pvc'

# path settings
model_path = 'weights/dpt_hybrid_nyu-2ce69ec7.pt'
dataset_path = 'video_inference_common/resources'
logs_path = 'train-logs'

checkpoint_name = 'version_290'

if k8s:
    model_path = os.path.join(k8s_pvc, 'dpt-hybrid-nyu.pt')
    dataset_path = os.path.join(k8s_repo, dataset_path)
    logs_path = os.path.join(k8s_pvc, logs_path)
    
    checkpoint_dir = os.path.join(logs_path, 'finetune', checkpoint_name, 'checkpoints')
    
    os.chdir('/')

In [6]:
# select a checkpoint path based on the ones available
checkpoint_dir = 'lightning_logs/version_3/checkpoints'
ckpts = os.listdir(checkpoint_dir)
sorted(ckpts, key=lambda x: int(re.findall(r'\d+', x)[0]))

[]

In [7]:
chosen_ckpt = ckpts[0]
checkpoint_path = os.path.join(checkpoint_dir, chosen_ckpt)

IndexError: list index out of range

In [ ]:
model = InteriorNetDPT(batch_size=1, 
                       lr=0, 
                       num_epochs=0, 
                       model_path=checkpoint_path)

model.to(device)
model.freeze()

In [8]:
dataset = InteriorNetDataset(dataset_path, split='val', 
                             transform='default', subsample=True)

loader = DataLoader(dataset, batch_size=1)

metrics = DepthMetrics()

epoch_num = re.findall(r'\d+', chosen_ckpt)[0]
if f'st-{epoch_num}.pt' in os.listdir(os.path.dirname(checkpoint_dir)):
    st = torch.load(os.path.join(os.path.dirname(checkpoint_dir), f'st-{epoch_num}.pt'))
    if st['s'][0].numel() == 1:
        s,t = torch.tensor(st['s']).mean(0), torch.tensor(st['t']).mean(0)
    else:
        s,t = torch.cat(st['s']).reshape(-1, *list(st['s'][0].shape)), torch.cat(st['t']).reshape(-1, *list(st['t'][0].shape))
    s,t = s.to(device), t.to(device)
else:
    st = None

NameError: name 'chosen_ckpt' is not defined

In [ ]:
st

In [ ]:
with torch.no_grad():
    for batch in loader:
        im, depth = batch['image'].to(device), batch['depth'].to(device)
        out = model(im)
        
#         print(out.mean())
#         acc = metrics(out, depth, st=(s,t) if st else None) # val_
#         print(acc)

        plt.imshow(im.squeeze(0).permute([1,2,0]).cpu())
        plt.show()
        plt.imshow(depth.squeeze(0).cpu())
        plt.show()
        plt.imshow(out.squeeze(0).cpu())
        plt.show()
        break

In [ ]:
# with torch.no_grad():
#     for i in np.random.randint(0,1000,10):
#         sample = interiornet_dataset[i]
#         image, depth = torch.tensor(sample['image']).to(device), 1/sample['depth']
#         out = model(image.unsqueeze(0)).squeeze(0).to('cpu')
              
#         fig,axs = plt.subplots(1,2)
#         axs[0].imshow((out - out.min()) / (out.max()-out.min()))
#         axs[1].imshow(depth)
#         plt.show()
        
#         break

In [4]:
%%capture
model = InteriorNetDPT(1,0,0,model_path, extractor='rn18')
# original = InteriorNetDPT(1,0,0,model_path)

model.to(device)
# original.to(device)

In [5]:
x = torch.ones((1,3,640,480)).to(device)

In [6]:
z = model(x)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


{'3': tensor([[[ 9.2299e-01,  9.3345e-01,  1.0393e+00,  ..., -8.1179e-01,
           1.6876e-01,  7.9964e-01],
         [ 4.9796e-01, -5.8956e+00, -3.8405e+00,  ..., -2.1222e+00,
          -2.1699e-01,  1.6136e+00],
         [ 4.9839e-01, -5.8869e+00, -3.8635e+00,  ..., -2.1348e+00,
          -2.1725e-01,  1.6256e+00],
         ...,
         [ 4.4820e-01,  2.3224e-01,  1.1890e-01,  ..., -2.2903e-01,
           4.7740e-01, -1.3227e-01],
         [ 4.4474e-01,  2.2313e-01,  8.4456e-02,  ..., -1.7782e-01,
           4.9974e-01, -3.7022e-02],
         [ 4.5588e-01,  2.1189e-01,  8.5682e-02,  ..., -1.6214e-01,
           4.8760e-01, -4.6805e-03]]], device='cuda:0', grad_fn=<AddBackward0>), '4': tensor([[[ 7.9636e-01,  2.0049e+00,  8.5451e-01,  ..., -3.2847e-01,
          -2.6496e-01,  1.8031e+00],
         [ 2.5069e-01, -5.8533e+00, -3.2051e+00,  ..., -1.8061e+00,
          -4.4380e-01,  2.1010e+00],
         [ 2.4888e-01, -5.8471e+00, -3.2301e+00,  ..., -1.8192e+00,
          -4.4310e-01, 

KeyError: '1'